In [ ]:
#Run this cell once per session ideally
#Run once to mount the link to Drive on YOUR implementation of Google Colab
from google.colab import drive
drive.mount('/content/drive') #if you put the Fall20_IntrotoDS_Final_Project folder in a subdirectory of My Drive, the paths won't work, make sure to add shared folder to your Drive
!pip install biopython
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Bio #biopython package name
from Bio import Entrez
from os import chdir,getcwd

Mounted at /content/drive
     |████████████████████████████████| 2.3MB 5.6MB/s 


In [ ]:
#Step 1: Execute query to get PMIDs.
def get_ids_from_term(query):
    Entrez.email = 'katherine.wall@uth.tmc.edu' #https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch for esearch parameters
    handle = Entrez.esearch(db='pubmed',
                            id='id',
                            retmax='10000000', #only pulls up to 10 million results at a time
                            rettype='gb',
                            retmode='xml',
                            datetype='pdat',
                            mindate='2020/01/01',
                            maxdate='2020/09/01',
                            field='All Fields', #MeSH Field is Medical Subject Headings, also being returned
                            term=query,)
    results = Entrez.read(handle)
    return results

#id_list = get_ids_from_term('HIV')['IdList']
#id_list

['33170905', '33170882', '33170860', '33170840', '33169719', '33169084', '33166373', '33166363', '33166351', '33166312', '33165413', '33165341', '33163396', '33163291', '33163290', '33163258', '33163060', '33163028', '33163024', '33163022', '33163021', '33163020', '33163019', '33163018', '33163017', '33163016', '33163015', '33162998', '33162972', '33162863', '33162756', '33162711', '33154978', '33156877', '33156871', '33156846', '33156765', '33156764', '33156763', '33156759', '33156757', '33156755', '33156754', '33154753', '33154260', '33152053', '33151997', '33150233', '33150232', '33150227', '33150054', '33149767', '33149744', '33149698', '33149697', '33149631', '33149599', '33148833', '33147295', '33147285', '33147250', '33147218', '33146325', '33146301', '33146297', '33145530', '33145479', '33145422', '33145207', '33145145', '33144828', '33143525', '32954217', '33140938', '33138958', '33137166', '33137103', '33134503', '33134195', '33134185', '33133222', '33133192', '33133191', '33

In [ ]:
#Step 2: Split PMID list into sections of 200 IDs.
def chunks(List, n): #chunk the list because only can submit 200 ids at a time to efetch
  for i in range(0, len(List), n):
    yield List[i:i + n]

# id_chunks = list(chunks(id_list, 200))
# print(id_chunks) #id_list has been chunked into sections of 200.
# print('Chunk Length:',len(id_chunks[0]))
# print('Number of Chunks in List',len(id_chunks))
# chunkj = ','.join(id_chunks[0]) #['33152053', '33151997', '33150233', '3315...'] TO '32081778,32267890,20348614'
# chunkj

[['33170905', '33170882', '33170860', '33170840', '33169719', '33169084', '33166373', '33166363', '33166351', '33166312', '33165413', '33165341', '33163396', '33163291', '33163290', '33163258', '33163060', '33163028', '33163024', '33163022', '33163021', '33163020', '33163019', '33163018', '33163017', '33163016', '33163015', '33162998', '33162972', '33162863', '33162756', '33162711', '33154978', '33156877', '33156871', '33156846', '33156765', '33156764', '33156763', '33156759', '33156757', '33156755', '33156754', '33154753', '33154260', '33152053', '33151997', '33150233', '33150232', '33150227', '33150054', '33149767', '33149744', '33149698', '33149697', '33149631', '33149599', '33148833', '33147295', '33147285', '33147250', '33147218', '33146325', '33146301', '33146297', '33145530', '33145479', '33145422', '33145207', '33145145', '33144828', '33143525', '32954217', '33140938', '33138958', '33137166', '33137103', '33134503', '33134195', '33134185', '33133222', '33133192', '33133191', '3

In [ ]:
#Step 3: On 200 IDs at a time, pull an extraction using Entrez efetch
def execute_search_on_ids(idlist):
  Entrez.email = 'katherine.wall@uth.tmc.edu'
  Entrez.api_key = '7d4a578699b1fdb8728e88125d3e65ae3708' #unique to my NCBI account, see post https://stackoverflow.com/questions/53951963/how-to-solve-http-error-429-in-biopython
  handle = Entrez.efetch(db="pubmed", id=idlist, retmode='xml')
  record = Entrez.read(handle)
  retrieval = record['PubmedArticle']
  return retrieval

# id_check = '32081778,32267890,20348614' #200 at a time 
# retrieval = execute_search_on_ids(chunkj)
# retrieval[0:5]

[{'MedlineCitation': DictElement({'OtherAbstract': [], 'OtherID': [], 'GeneralNote': [], 'CitationSubset': ['IM'], 'KeywordList': [], 'SpaceFlightMission': [], 'PMID': StringElement('33170905', attributes={'Version': '1'}), 'DateRevised': {'Year': '2020', 'Month': '11', 'Day': '10'}, 'Article': DictElement({'Language': ['eng'], 'ArticleDate': [DictElement({'Year': '2020', 'Month': '11', 'Day': '10'}, attributes={'DateType': 'Electronic'})], 'ELocationID': [StringElement('10.1371/journal.pone.0242068', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})], 'Journal': {'ISSN': StringElement('1932-6203', attributes={'IssnType': 'Electronic'}), 'JournalIssue': DictElement({'Volume': '15', 'Issue': '11', 'PubDate': {'Year': '2020'}}, attributes={'CitedMedium': 'Internet'}), 'Title': 'PloS one', 'ISOAbbreviation': 'PLoS One'}, 'ArticleTitle': 'Hematologic alterations and early mortality in a cohort of HIV positive African patients.', 'Pagination': {'MedlinePgn': 'e0242068'}, 'Abstract': {'Abstract

In [ ]:
#Step 4: Parse the returned XML object to get lists of title, dates, etc for the 200 ids
def paper_infov2(retrieval,chunkj): 
  id = []
  titles = []
  authors = []
  pubdates = []
  abs = []
  for i in range(len(retrieval)):
    id.append(dict(retrieval[i]['MedlineCitation'])['PMID'].split()[0])

    title = dict(dict(retrieval[i]['MedlineCitation'])['Article'])['ArticleTitle']
    titles.append(title)
    
    try:
      names = pd.DataFrame(list(dict(dict(retrieval[i]['MedlineCitation'])['Article'])['AuthorList']))
      names['combined'] = names.ForeName+' '+names.LastName
      listednames = names.combined.to_list()
      authors.append(listednames)
    except KeyError:
      authors.append('Error: no author list found')
    except AttributeError:
      authors.append('Error: either first name or last name was not found')

    pubdate = dict(dict(dict(dict(retrieval[i]['MedlineCitation'])['Article'])['Journal'])['JournalIssue'])['PubDate']
    pubdates.append(pubdate)

    try:
      full_text = dict(dict(dict(retrieval[i]['MedlineCitation'])['Article'])['Abstract'])['AbstractText']
      abstract_text = []
      for k in range(len(full_text)):
        abstract_text += full_text[k].split(', attributes=')
      abs.append(abstract_text)
    except KeyError:
      abs.append('Error: no abstract text found')
  return id, titles, authors, pubdates, abs


In [ ]:
#FINAL RESULT, takes a couple of minutes to run
def extraction(query):
  ID,TITLE,AUTH,PDAT,ABS = [],[],[],[],[]
  returned = get_ids_from_term(query) #Step1
  ids = returned['IdList']
  id_chunks = list(chunks(ids, 200)) #Step2
  for i in id_chunks: 
    chunkj = ','.join(i) #['33152053', '33151997'] TO '33152053,33151997'
    retrieval = execute_search_on_ids(chunkj) #Step 3 Get entrez xml object for 200 IDs
    id, titles, authors, pubdates, abs = paper_infov2(retrieval, chunkj) #Step 4 Parse xml object
    ID += id
    TITLE += titles
    AUTH += authors
    PDAT += pubdates
    ABS += abs
  all_data = {'PMID':ID,'Title':TITLE,'AuthorList':AUTH,'PubDate':PDAT,'AbtractText':ABS}
  return all_data

# final_result = extraction('HIV')
# combined_df = pd.DataFrame(final_result)
# combined_df

,PMID,Title,AuthorList,PubDate,AbtractText
0,33170905,Hematologic alterations and early mortality in...,"[Fausto Ciccacci, Francesca Lucaroni, Roberto ...",{'Year': '2020'},[Infection with Human Immunodeficiency Virus (...
1,33170882,The magnitude of anemia and associated factors...,"[Nigus Alemu Hailu, Tesfaye Tolessa, Zenawi Ha...",{'Year': '2020'},[Patients with Diabetic Mellitus are at higher...
2,33170860,In-vitro inhibitory effect of maternal breastm...,"[Katayi Mwila Kazimbaya, Caroline C Chisenga, ...",{'Year': '2020'},[Despite contributing to a significant reducti...
3,33170840,National-level effectiveness of ART to prevent...,"[Andrew Agabu, Andrew L Baughman, Christa Fisc...",{'Year': '2020'},[Namibia introduced the prevention of mother t...
4,33169719,Estimating the Loss in Expectation of Life and...,"[Navisa Sadat Seyedghasemi, Abbas Bahrampour, ...","{'Year': '2020', 'Month': 'Aug', 'Day': '03'}",[Information regarding the prognosis and burde...
...,...,...,...,...,...
12915,28523394,Introduction: Framing 'Post-AIDS' and Global H...,[Gráinne O'Connell],"{'Year': '2020', 'Month': 'Jun'}","[This special issue, entitled ""Post-AIDS' and ..."
12916,28389836,Temporal Dissonance: South African Historians ...,[Carla Tsampiras],"{'Year': '2020', 'Month': 'Jun'}",[While foregrounding the historiography of HIV...
12917,28315040,"HIV Stigma, Gay Identity, and Caste 'Untouchab...",[Shamira A Meghani],"{'Year': '2020', 'Month': 'Jun'}",[In this article I read textual metaphors of '...
12918,28243819,HIV and AIDS in Irish Theatre: Queer Masculini...,[Cormac O'Brien],"{'Year': '2020', 'Month': 'Jun'}",[This essay provides a critical survey of key ...


In [ ]:
#Save to drive archive folder
# combined_df.to_csv('/content/drive/MyDrive/Fall20_IntrotoDS_Final_Project/Archive/someonelseisrunning.csv',index=False)